### Import modules

In [12]:
import json
import os

import boto3
from botocore.config import Config
from opensearchpy import (AWSV4SignerAuth, OpenSearch, RequestsHttpConnection,
                          helpers)
from opensearchpy.exceptions import RequestError

### Initialize and configure Boto Client for OpenSearch Serverless

In [13]:
boto_config = Config(
    region_name = 'us-east-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

In [14]:
session = boto3.Session(profile_name="rapid-innovation-dev")
client = session.client("opensearchserverless",config=boto_config)

### Define encryption policy

In [15]:
policy = """{
                "Rules": [
                    {
                        "Resource": [
                            "collection/oscars-collection"
                        ],
                        "ResourceType": "collection"
                    }
                ],
                "AWSOwnedKey": true
            }"""

try:
    response = client.create_security_policy(
        description="oscars collection encryption security policy",
        name="oscars-encryption-policy",
        policy=policy,
        type="encryption",
    )
    print(json.dumps(response, indent=2))
except client.exceptions.ConflictException as ex:
    print("oscars-encryption-policy exists")

oscars-encryption-policy exists


### Define network policy

In [16]:
policy = """[
                {
                    "Rules": [
                        {
                            "Resource": [
                                "collection/oscars-collection"
                            ],
                            "ResourceType": "collection"
                        }
                    ],
                    "AllowFromPublic": true
                }
            ]"""

try:
    response = client.create_security_policy(
        description="oscars collection network security policy",
        name="oscars-network-policy",
        policy=policy,
        type="network",
    )
    print(json.dumps(response, indent=2))
except client.exceptions.ConflictException as ex:
    print("oscars-network-policy already exists")

oscars-network-policy already exists


### Define data access policy

In [19]:
#replace the IAM principle with your own
policy = """[
                {
                    "Rules": [
                        {
                            "Resource": [
                                "collection/*"
                            ],
                            "Permission": [
                                "aoss:*"
                            ],
                            "ResourceType": "collection"
                        },
                        {
                            "Resource": [
                                "index/*/*"
                            ],
                            "Permission": [
                                "aoss:*"
                            ],
                            "ResourceType": "index"
                        }
                    ],
                    "Principal": [
                        "arn:aws:sts::683883881884:assumed-role/AWSReservedSSO_rapid-innovation-admin_eeb907705f3c7f8e/vr@presidio.com"
                    ],
                    "Description": "Rule 1"
                }
            ]"""

try:
    response = client.create_access_policy(
        description="oscars collection data access policy",
        name="oscars-data-access-policy",
        policy=policy,
        type="data"
    )
    print(json.dumps(response, indent=2))
except client.exceptions.ConflictException as ex:
    print("oscars-data-access-policy already exists")

oscars-data-access-policy already exists


### Create the collection

In [20]:
# create vector search collection
try:
    response = client.create_collection(
        description="oscars collection",
        name="oscars-collection",
        type="VECTORSEARCH"
    )
    print(json.dumps(response, indent=2))
except client.exceptions.ConflictException as ex:
    print("oscars-collection already exists")

oscars-collection already exists


In [21]:
while True:
    response = client.batch_get_collection(names=['oscars-collection'])
    collection_details = response["collectionDetails"][0]
    if collection_details.get('collectionEndpoint', None):
        end_point = response["collectionDetails"][0]['collectionEndpoint']
        host = end_point.split("https://")[-1]
        print(response)
        print(host)
        with open('host.txt', 'w+') as fp:
            fp.write(host)
        break

{'collectionDetails': [{'arn': 'arn:aws:aoss:us-east-1:683883881884:collection/w6yidk747fs2m0t09vaj', 'collectionEndpoint': 'https://w6yidk747fs2m0t09vaj.us-east-1.aoss.amazonaws.com', 'createdDate': 1718281481578, 'dashboardEndpoint': 'https://w6yidk747fs2m0t09vaj.us-east-1.aoss.amazonaws.com/_dashboards', 'description': 'oscars collection', 'id': 'w6yidk747fs2m0t09vaj', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1718282074091, 'name': 'oscars-collection', 'standbyReplicas': 'ENABLED', 'status': 'ACTIVE', 'type': 'VECTORSEARCH'}], 'collectionErrorDetails': [], 'ResponseMetadata': {'RequestId': '2fc4c881-bb19-4362-a0a4-e147a9f5abad', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2fc4c881-bb19-4362-a0a4-e147a9f5abad', 'date': 'Mon, 17 Jun 2024 09:58:49 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '539', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
w6yidk747fs2m0t09vaj.us-east-1.aoss.amazonaws.com


### Initialize and configure OpenSearch client

In [22]:
host = host #replace this with the value from the AWS Management Console
region = "us-east-1"
service = "aoss"
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

In [23]:
client = OpenSearch(
    hosts = [{"host": host, "port": 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

### Create an index

In [24]:
index_name = "oscars-index"
index_body = {
    "mappings": {
        "properties": {
            "nominee_text": {"type": "text"},
            "nominee_metadata": {"type": "text"},
            "nominee_vector": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "engine": "nmslib",
                    "space_type": "cosinesimil",
                    "name": "hnsw",
                    "parameters": {"ef_construction": 512, "m": 16},
                },
            },
        }
    },
    "settings": {
        "index": {
            "number_of_shards": 2,
            "knn.algo_param": {"ef_search": 512},
            "knn": True,
        }
    },
}

try:
    response = client.indices.create(index_name, body=index_body)
    print(json.dumps(response, indent=2))
except RequestError as ex:
    print("oscars-index already exists")

oscars-index already exists


### Verify the index

In [25]:
try:
    response = client.indices.get("oscars-index")
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

{
  "oscars-index": {
    "aliases": {},
    "mappings": {
      "properties": {
        "nominee_metadata": {
          "type": "text"
        },
        "nominee_text": {
          "type": "text"
        },
        "nominee_vector": {
          "type": "knn_vector",
          "dimension": 1536,
          "method": {
            "engine": "nmslib",
            "space_type": "cosinesimil",
            "name": "hnsw",
            "parameters": {
              "ef_construction": 512,
              "m": 16
            }
          }
        }
      }
    },
    "settings": {
      "index": {
        "number_of_shards": "2",
        "knn.algo_param": {
          "ef_search": "512"
        },
        "provided_name": "oscars-index",
        "knn": "true",
        "creation_date": "1718614042160",
        "number_of_replicas": "0",
        "uuid": "PzphJZAB-3YzZxf4JXq4",
        "version": {
          "created": "135217827"
        }
      }
    }
  }
}
